In [4]:
# 1. Импорты и «константы»
from pathlib import Path, PurePosixPath
import time, torch, json, os, gc
from datasets import load_dataset, Audio
from transformers import (AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline)
from optimum.onnxruntime import (ORTModelForSpeechSeq2Seq,
                                 ORTQuantizer, AutoQuantizationConfig)

BASE_ID   = "openai/whisper-large-v3"      # исходная модель
ONNX_DIR  = Path("whisper_large_v3_onnx")   # куда сохраняем export
QN_DIR    = Path("whisper_large_v3_int8")   # куда сохраняем INT8

device = "cuda"
dtype  = torch.float16          # fp16 на 4090 самое выгодное
SAMPLES = 10                    # сколько аудио берём для WER/тайминга

In [5]:
# ── Cell 2: базовые импорты и GPU инфо ────────────────────────────────────────
import torch, time, gc, psutil, os, json, numpy as np
from transformers import (AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline,
                          GenerationConfig)
from datasets import load_dataset, Audio, disable_caching
from evaluate import load as load_metric

device = "cuda:0"
print("Torch:", torch.__version__, "| GPU:", torch.cuda.get_device_name(0))
disable_caching()  # экономим место в ~/.cache/huggingface

Torch: 2.7.0+cu126 | GPU: NVIDIA GeForce RTX 4090


In [6]:
# ── Новый кусочек вместо предыдущей Cell-3 ────────────────────────────────
from datasets import load_dataset, DownloadConfig, Audio

download_config = DownloadConfig(max_retries=10, resume_download=True)

stream_ds = load_dataset(
    "librispeech_asr",
    "clean",
    split="test",
    streaming=True,
    download_config=download_config,
)

stream_ds = stream_ds.cast_column("audio", Audio(sampling_rate=16_000))
ds = list(stream_ds.take(30))          # превращаем в список обычных dict’ов
print(f"Взято {len(ds)} примеров")
print(ds[0].keys())

Взято 30 примеров
dict_keys(['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'])


In [7]:
# ── Cell 4: helper-функции замера (ИСПРАВЛЕНО v2) ───────────────────────────
import time, numpy as np, torch, gc, psutil, os, json
from evaluate import load as load_metric
from transformers import pipeline

# Убедитесь, что 'device' определен глобально перед вызовом asr_pipe
# Например: device = "cuda:0" if torch.cuda.is_available() else "cpu"
# или как в вашей ячейке 2: device = "cuda:0"

wer_metric = load_metric("wer")

def asr_pipe(model, processor, dtype=torch.float16):
    # Используем device, определенный для GPU (или CPU, если нет GPU)
    current_device = device # Захватываем значение device
    print(f"Создание пайплайна на устройстве: {current_device}")
    return pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        device=current_device,
        torch_dtype=dtype,
    )

# def evaluate_model(p, dataset, max_items=None):
#     refs, preds, lat = [], [], []
#     iterable = dataset if max_items is None else dataset[:max_items]
#     print(f"Оценка модели на {p.device}...") # Добавим лог устройства пайплайна

#     for i, sample in enumerate(iterable):
#         audio_dict = sample["audio"]

#         try:
#             wav = audio_dict["array"]
#             sr = audio_dict["sampling_rate"]

#             if not isinstance(wav, np.ndarray):
#                 print(f"Предупреждение: Ожидался np.ndarray, получен {type(wav)} в sample {i}. Пропускаем.")
#                 continue
#             if sr != 16_000:
#                  print(f"Предупреждение: Неожиданная частота дискретизации {sr} != 16000 в sample {i}.")
#                  # Но мы все равно передаем массив как есть, т.к. cast_column должен был отработать

#         except KeyError as e:
#             print(f"Ошибка: Отсутствует ключ {e} в audio_dict для sample {i}. Структура данных: {audio_dict.keys()}")
#             continue
#         except Exception as e:
#             print(f"Неожиданная ошибка при извлечении аудио из sample {i}: {e}")
#             continue

#         start = time.time()
#         try:
#             # --- ИЗМЕНЕНИЕ ЗДЕСЬ ---
#             # Убираем sampling_rate=sr, т.к. массив wav уже имеет нужную частоту (16kHz)
#             # и пайплайн, похоже, не ожидает этот параметр в данном контексте.
#             result = p(wav)
#             # --- КОНЕЦ ИЗМЕНЕНИЯ ---
#             text = result["text"]
#         except Exception as e:
#             # Добавим вывод типа пайплайна для диагностики
#             print(f"Ошибка во время выполнения пайплайна ({type(p).__name__}) для sample {i}: {e}")
#             print(f"Тип wav: {type(wav)}, форма: {wav.shape if isinstance(wav, np.ndarray) else 'N/A'}, sr (из данных): {sr}")
#             # Устанавливаем result в None или пустой dict, чтобы del не падал
#             result = None
#             text = "" # Или другое значение по умолчанию

#         lat.append(time.time() - start)

#         refs.append(sample["text"].lower())
#         preds.append(text.lower())

#         # Очистка памяти
#         # Проверяем существование result перед удалением
#         del wav, audio_dict
#         if result is not None:
#              del result
#         if i % 5 == 0:
#              if torch.cuda.is_available() and hasattr(p, 'device') and p.device.type == 'cuda':
#                  torch.cuda.empty_cache()
#         gc.collect()


#     wer_score = None
#     if preds and refs:
#          valid_preds = [p if isinstance(p, str) else "" for p in preds]
#          valid_refs = [r if isinstance(r, str) else "" for r in refs]
#          if valid_preds and valid_refs:
#               try:
#                   wer_score = wer_metric.compute(predictions=valid_preds, references=valid_refs)
#               except Exception as e:
#                   print(f"Ошибка при вычислении WER: {e}")
#                   print(f"Preds: {valid_preds[:5]}") # Показать начало списков для отладки
#                   print(f"Refs: {valid_refs[:5]}")
#                   wer_score = -1 # Или другое значение-индикатор ошибки


#     return {
#         "wer": wer_score,
#         "latency_s": np.mean(lat) if lat else 0,
#         "processed_items": len(preds)
#     }

# --- Напоминание ---
# После изменения Cell 4, обязательно перезапустите ее,
# а затем перезапустите Cell 5 (где создается pipe_fp16 и вызывается evaluate_model).
# Затем можно будет переходить к Cell 6 и 7.

In [9]:
# ── Cell 5: FP16 Whisper-large-v3, базовый прогон ─────────────────────────────
model_id = "openai/whisper-large-v3"
torch_dtype = torch.float16

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)
processor = AutoProcessor.from_pretrained(model_id)

pipe_fp16 = asr_pipe(model, processor, dtype=torch_dtype)

stats_fp16 = evaluate_model(pipe_fp16, ds, max_items=10)
print("FP16:", stats_fp16)

Device set to use cuda:0
/home/ubuntu/miniconda3/envs/ort-gpu/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Создание пайплайна на устройстве: cuda:0
Оценка модели (WhisperForConditionalGeneration) на устройстве: cuda:0...
Прогревочных запусков: 3, Оценочных запусков: 10
Выполнение прогревочных запусков...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Прогрев завершен.
Начало оценочных запусков (10 сэмплов)...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Обработано 10/10 оценочных сэмплов...
Оценочные запуски завершены.
FP16: {'wer': 0.10047846889952153, 'latency_s': 0.6250935133999918, 'latency_std_s': 0.42636415074513684, 'processed_items': 10, 'peak_ram_mb': '796.8', 'peak_vram_mb': '3208.4'}


In [7]:
# ── Cell 6 (фикс) ────────────────────────────────────────────────────────────
from optimum.onnxruntime import (
    ORTModelForSpeechSeq2Seq, ORTQuantizer, AutoQuantizationConfig
)
from pathlib import Path
import shutil, os

export_dir_fp32 = "whisper_onnx_fp32"
export_dir_int8 = "whisper_onnx_int8"

# if not Path(export_dir_fp32).exists():
#     model_ort = ORTModelForSpeechSeq2Seq.from_pretrained(model_id, export=True)
#     shutil.move(model_ort.model_save_dir, export_dir_fp32)
# else:
#     print("ONNX-FP32 уже существует")

# qconfig = AutoQuantizationConfig.avx512_vnni(
#     is_static=False,
#     per_channel=False,
#     operators_to_quantize=["MatMul"]   # ← Квантуем только матмулы
# )

# onnx_files = list(Path(export_dir_fp32).glob("*.onnx"))
# print("Квантуем:", [f.name for f in onnx_files])

# os.makedirs(export_dir_int8, exist_ok=True)

# for onnx_path in onnx_files:
#     quantizer = ORTQuantizer.from_pretrained(export_dir_fp32,
#                                              file_name=onnx_path.name)
#     quantizer.quantize(
#         save_dir            = export_dir_int8,
#         quantization_config = qconfig,
#     )

# for fname in ["config.json", "generation_config.json"]:
#     src = Path(export_dir_fp32) / fname
#     if src.exists():
#         shutil.copy(src, export_dir_int8)

# print("INT-8-модель готова →", export_dir_int8)

In [8]:
# ── Cell 7 (правка): загрузка INT8-модели на CPU и оценка ───────────────────
from transformers import pipeline
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq

model_int8 = ORTModelForSpeechSeq2Seq.from_pretrained(
    export_dir_int8,
    provider="CPUExecutionProvider",   # ключевая строка
)
pipe_int8 = pipeline(
    "automatic-speech-recognition",
    model=model_int8,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=-1,                         # -1 = CPU
)

stats_int8 = evaluate_model(pipe_int8, ds, max_items=10)
print("INT8-CPU:", stats_int8)

Device set to use cpu


Оценка модели на cpu...
INT8-CPU: {'wer': 0.02390438247011952, 'latency_s': 5.581054902076721, 'processed_items': 10}


In [8]:
# ── Cell 9: Обновленная evaluate_model с прогревом и точным замером времени ──
import time, numpy as np, torch, gc, psutil, os, json
from evaluate import load as load_metric
from transformers import pipeline, Pipeline # Убедимся, что Pipeline импортирован

# --- Функция замера памяти (остается без изменений) ---
def get_memory_usage():
    process = psutil.Process(os.getpid())
    ram_mb = process.memory_info().rss / (1024**2)
    vram_mb = 0
    try:
        if torch.cuda.is_available() and torch.cuda.is_initialized():
            vram_mb = torch.cuda.memory_allocated(0) / (1024**2)
    except Exception:
        pass
    return ram_mb, vram_mb

# --- Обновленная evaluate_model ---
wer_metric = load_metric("wer")

def evaluate_model(p: Pipeline, dataset, max_items=None, num_warmup_runs=3): # Добавлен параметр num_warmup_runs
    refs, preds = [], []
    # Переносим замер задержки внутрь основного цикла, чтобы он был более чистым
    pure_latencies = [] # Список для чистых задержек вызова пайплайна

    iterable = dataset if max_items is None else dataset
    # Если max_items задан, нам нужно взять warmup_runs + max_items из датасета
    # чтобы иметь достаточно данных и для прогрева, и для оценки.
    total_samples_needed = (num_warmup_runs if max_items is not None else 0) + \
                           (max_items if max_items is not None else len(dataset))

    # Ограничиваем iterable, если dataset это список и мы используем max_items
    if isinstance(dataset, list) and total_samples_needed < len(dataset):
        effective_iterable = dataset[:total_samples_needed]
    else:
        # Для потоковых датасетов или если нужно меньше, чем есть
        effective_iterable = list(iterable.take(total_samples_needed)) if hasattr(iterable, 'take') else list(iterable)[:total_samples_needed]

    if len(effective_iterable) < (num_warmup_runs if max_items is not None else 0) + \
                                 (max_items if max_items is not None else 1):
        print(f"Предупреждение: Недостаточно данных для {num_warmup_runs} прогревочных запусков и {max_items} оценочных. "
              f"Доступно: {len(effective_iterable)}. Прогрев может быть сокращен или отсутствовать.")
        # Корректируем num_warmup_runs, если max_items не задан или данных мало
        if max_items is None:
             actual_eval_items = max(1, len(effective_iterable) - num_warmup_runs)
        else:
             actual_eval_items = max_items
        # Уменьшаем прогрев, если не хватает данных
        effective_num_warmup_runs = max(0, len(effective_iterable) - actual_eval_items)
    else:
        effective_num_warmup_runs = num_warmup_runs

    print(f"Оценка модели ({type(p.model).__name__}) на устройстве: {p.device}...")
    print(f"Прогревочных запусков: {effective_num_warmup_runs}, Оценочных запусков: {max_items if max_items is not None else len(effective_iterable) - effective_num_warmup_runs}")

    # --- Замеры памяти (как и ранее) ---
    initial_ram, initial_vram = get_memory_usage()
    max_ram_seen = initial_ram
    peak_vram_torch = 0
    is_torch_model_on_gpu = False
    if hasattr(p, 'model') and isinstance(p.model, torch.nn.Module) and p.device.type == 'cuda':
        try:
            torch.cuda.reset_peak_memory_stats(p.device)
            is_torch_model_on_gpu = True
        except Exception: pass

    # --- Прогревочные запуски ---
    if effective_num_warmup_runs > 0:
        print("Выполнение прогревочных запусков...")
        for i in range(effective_num_warmup_runs):
            if i >= len(effective_iterable): break # Предохранитель
            sample = effective_iterable[i]
            audio_dict = sample["audio"]
            try:
                wav = audio_dict["array"]
                sr = audio_dict["sampling_rate"]
                if not isinstance(wav, np.ndarray) or sr != 16_000: continue
                _ = p(wav) # Вызываем пайплайн, результат не важен
            except Exception as e:
                print(f"Ошибка на прогревочном запуске {i+1}: {e}")
                continue # Продолжаем, даже если прогрев упал
        print("Прогрев завершен.")

    # --- Основной цикл оценки (только для замера времени и WER) ---
    # Определяем срез данных для реальной оценки
    eval_samples_start_index = effective_num_warmup_runs
    num_eval_items = max_items if max_items is not None else (len(effective_iterable) - effective_num_warmup_runs)
    eval_samples_end_index = eval_samples_start_index + num_eval_items

    print(f"Начало оценочных запусков ({num_eval_items} сэмплов)...")
    for i in range(eval_samples_start_index, eval_samples_end_index):
        if i >= len(effective_iterable): break # Предохранитель
        sample = effective_iterable[i]
        audio_dict = sample["audio"]

        try:
            wav = audio_dict["array"]
            sr = audio_dict["sampling_rate"]
            if not isinstance(wav, np.ndarray) or sr != 16_000:
                print(f"Предупреждение: Некорректные данные в оценочном sample {i}. Пропускаем.")
                continue
        except KeyError as e:
            print(f"Ошибка: Отсутствует ключ {e} в audio_dict для оценочного sample {i}.")
            continue
        except Exception as e:
            print(f"Неожиданная ошибка при извлечении аудио из оценочного sample {i}: {e}")
            continue

        # --- Точный замер времени выполнения пайплайна ---
        torch.cuda.synchronize() if p.device.type == 'cuda' else None # Для точного замера на GPU
        start_time = time.perf_counter()
        try:
            result = p(wav)
            text = result["text"]
        except Exception as e:
            print(f"Ошибка во время выполнения пайплайна ({type(p).__name__}) для оценочного sample {i}: {e}")
            result = None
            text = ""
        torch.cuda.synchronize() if p.device.type == 'cuda' else None # Для точного замера на GPU
        end_time = time.perf_counter()
        pure_latencies.append(end_time - start_time)
        # --- Конец точного замера ---

        refs.append(sample["text"].lower())
        preds.append(text.lower())

        # Обновляем пиковые значения памяти (делаем это для каждого сэмпла, включая оценочные)
        current_ram, _ = get_memory_usage() # VRAM от PyTorch statistics будет точнее для PyTorch моделей
        max_ram_seen = max(max_ram_seen, current_ram)

        del wav, audio_dict
        if result is not None: del result
        if (i - eval_samples_start_index + 1) % 10 == 0 : # Каждые 10 обработанных сэмплов
            print(f"Обработано {i - eval_samples_start_index + 1}/{num_eval_items} оценочных сэмплов...")
            gc.collect()
            if torch.cuda.is_available() and p.device.type == 'cuda':
                 torch.cuda.empty_cache()
    print("Оценочные запуски завершены.")

    if is_torch_model_on_gpu:
        try:
            peak_vram_torch = torch.cuda.max_memory_allocated(p.device) / (1024**2)
        except Exception: pass

    wer_score = None
    if preds and refs:
         valid_preds = [pred_text if isinstance(pred_text, str) else "" for pred_text in preds]
         valid_refs = [ref_text if isinstance(ref_text, str) else "" for ref_text in refs]
         if valid_preds and valid_refs:
              try:
                  wer_score = wer_metric.compute(predictions=valid_preds, references=valid_refs)
              except Exception as e:
                  print(f"Ошибка при вычислении WER: {e}")
                  wer_score = -1

    peak_ram_delta_mb = max_ram_seen - initial_ram
    peak_vram_mb_final = f"{peak_vram_torch:.1f}" if peak_vram_torch > 0 else 'N/A (Не PyTorch/GPU)'
    
    # Если это не PyTorch модель на GPU (например, ONNX на GPU), VRAM из torch.cuda.max_memory_allocated нерелевантен.
    # В этом случае, можно попробовать взять пик от get_memory_usage(), но он будет менее точным для GPU.
    if peak_vram_mb_final == 'N/A (Не PyTorch/GPU)' and p.device.type == 'cuda':
        # Для ONNX на GPU, пик VRAM от torch.cuda.max_memory_allocated() не работает.
        # Используем замер, сделанный после загрузки модели в Ячейке 11 (если доступен)
        # или указываем, что это общий пик во время выполнения.
        # Это значение уже передается в stats_int8_gpu['peak_vram_mb'] в Ячейке 11.
        # Здесь мы просто не переопределяем его, если оно уже установлено.
        pass


    return {
        "wer": wer_score,
        "latency_s": np.mean(pure_latencies) if pure_latencies else 0, # Используем pure_latencies
        "latency_std_s": np.std(pure_latencies) if pure_latencies else 0, # Стандартное отклонение задержек
        "processed_items": len(preds),
        "peak_ram_mb": f"{peak_ram_delta_mb:.1f}",
        "peak_vram_mb": peak_vram_mb_final
    }

print("Функция evaluate_model обновлена с прогревом и улучшенным замером времени.")

Функция evaluate_model обновлена с прогревом и улучшенным замером времени.


In [12]:
# ── Cell 10: Запуск FP16 на CPU ──────────────────────────────────────────────
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import gc

print("\n--- Запуск FP16 на CPU ---")
device_cpu = "cpu"
model_id = "openai/whisper-large-v3" # Убедимся, что ID модели доступен

# FP16 может быть очень медленным или неподдерживаемым на CPU для некоторых операций.
# Попробуем FP16, но если будут ошибки или слишком долго, переключимся на FP32.
# ВАЖНО: Для CPU обычно используют FP32 для лучшей совместимости и производительности.
fp16_on_cpu_dtype = torch.float16
# fp16_on_cpu_dtype = torch.float32 # Альтернатива, если FP16 не работает

stats_fp16_cpu = {}

try:
    print(f"Загрузка {model_id} ({fp16_on_cpu_dtype}) на {device_cpu}...")
    # Загружаем модель сразу на CPU
    model_fp16_cpu = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=fp16_on_cpu_dtype,
        low_cpu_mem_usage=False # low_cpu_mem_usage=True может потребовать GPU для метаданных
    ).to(device_cpu)

    # Процессор загружаем как обычно
    processor_fp16_cpu = AutoProcessor.from_pretrained(model_id)

    print("Создание пайплайна для FP16 на CPU...")
    pipe_fp16_cpu = pipeline(
        "automatic-speech-recognition",
        model=model_fp16_cpu,
        tokenizer=processor_fp16_cpu.tokenizer,
        feature_extractor=processor_fp16_cpu.feature_extractor,
        torch_dtype=fp16_on_cpu_dtype,
        device=-1, # Явно указываем CPU для пайплайна
    )

    print("Запуск оценки FP16 на CPU...")
    # Используем обновленную evaluate_model из Cell 9
    stats_fp16_cpu = evaluate_model(pipe_fp16_cpu, ds, max_items=SAMPLES) # Используем SAMPLES = 10
    print("\nРезультаты FP16 CPU:", stats_fp16_cpu)

except Exception as e:
    print(f"\nОшибка при выполнении FP16 на CPU ({fp16_on_cpu_dtype}): {e}")
    print("Возможно, стоит попробовать torch_dtype=torch.float32 или уменьшить max_items.")
    stats_fp16_cpu = {"error": str(e)} # Сохраняем информацию об ошибке

finally:
    # Очистка памяти
    if 'pipe_fp16_cpu' in locals(): del pipe_fp16_cpu
    if 'model_fp16_cpu' in locals(): del model_fp16_cpu
    if 'processor_fp16_cpu' in locals(): del processor_fp16_cpu
    gc.collect()

print("Завершено FP16 на CPU.")


--- Запуск FP16 на CPU ---
Загрузка openai/whisper-large-v3 (torch.float16) на cpu...


Device set to use cpu


Создание пайплайна для FP16 на CPU...
Запуск оценки FP16 на CPU...
Оценка модели (WhisperForConditionalGeneration) на устройстве: cpu...


/home/ubuntu/miniconda3/envs/ort-gpu/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



Результаты FP16 CPU: {'wer': 0.07569721115537849, 'latency_s': 72.75857982635497, 'processed_items': 10, 'peak_ram_mb': '2970.9', 'peak_vram_mb': 'N/A (Не PyTorch/GPU)'}
Завершено FP16 на CPU.


In [10]:
# ── Cell 11: Запуск INT8 ONNX на GPU ─────────────────────────────────────────
import torch
import gc
from pathlib import Path
from transformers import AutoProcessor, pipeline # Нужен процессор и пайплайн

print("\n--- Запуск INT8 ONNX на GPU ---")
stats_int8_gpu = {}
onnx_int8_dir = Path(export_dir_int8)
gpu_device_index = 0 # Используем GPU 0
gpu_device_name = f"cuda:{gpu_device_index}"

# Проверяем наличие ONNX Runtime GPU и папки с моделью
onnxruntime_gpu_available = False
try:
    import onnxruntime
    providers = onnxruntime.get_available_providers()
    print("Доступные провайдеры ONNX Runtime:", providers)
    if "CUDAExecutionProvider" in providers:
        onnxruntime_gpu_available = True
    else:
        print("\nПРЕДУПРЕЖДЕНИЕ: CUDAExecutionProvider не найден.")
        print("Убедитесь, что установлен пакет 'onnxruntime-gpu'.")
        print("pip install -U onnxruntime-gpu")
except ImportError:
    print("ПРЕДУПРЕЖДЕНИЕ: Библиотека onnxruntime не найдена.")

if not onnx_int8_dir.exists():
    print(f"ПРЕДУПРЕЖДЕНИЕ: Папка с INT8 ONNX моделью не найдена: {onnx_int8_dir}")

if onnxruntime_gpu_available and onnx_int8_dir.exists():
    try:
        from optimum.onnxruntime import ORTModelForSpeechSeq2Seq

        print(f"Загрузка INT8 ONNX модели из {onnx_int8_dir} на GPU...")
        model_int8_gpu = ORTModelForSpeechSeq2Seq.from_pretrained(
            onnx_int8_dir,
            provider="CUDAExecutionProvider",
            provider_options={'device_id': str(gpu_device_index)}, # Указываем ID устройства GPU
        )
        # Загрузим VRAM после загрузки модели (как прокси)
        vram_after_load_mb = 0
        if torch.cuda.is_available():
             vram_after_load_mb = torch.cuda.memory_allocated(gpu_device_index) / (1024**2)
             print(f"VRAM занято после загрузки INT8 ONNX модели: {vram_after_load_mb:.1f} MB")


        # Используем тот же процессор, что и для других моделей (если он еще существует)
        # Если нет, загружаем заново
        if 'processor' not in locals():
            print("Загрузка процессора...")
            processor = AutoProcessor.from_pretrained("openai/whisper-large-v3") # Исходный ID

        print("Создание пайплайна для INT8 ONNX на GPU...")
        pipe_int8_gpu = pipeline(
            "automatic-speech-recognition",
            model=model_int8_gpu,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            device=gpu_device_index, # Явно указываем GPU для пайплайна
        )

        print("Запуск оценки INT8 ONNX на GPU...")
        # Используем обновленную evaluate_model из Cell 9
        stats_int8_gpu = evaluate_model(pipe_int8_gpu, ds, max_items=SAMPLES) # Используем SAMPLES = 10

        # Добавим замер VRAM после загрузки в результаты, т.к. evaluate_model не может мерить пик ORT
        if 'peak_vram_mb' not in stats_int8_gpu or stats_int8_gpu['peak_vram_mb'] == 'N/A (Не PyTorch/GPU)':
             stats_int8_gpu['peak_vram_mb'] = f"~{vram_after_load_mb:.1f} (После загрузки)"

        print("\nРезультаты INT8 ONNX GPU:", stats_int8_gpu)

    except Exception as e:
        print(f"\nОшибка при выполнении INT8 ONNX на GPU: {e}")
        stats_int8_gpu = {"error": str(e)}

    finally:
        # Очистка памяти
        if 'pipe_int8_gpu' in locals(): del pipe_int8_gpu
        if 'model_int8_gpu' in locals(): del model_int8_gpu
        # 'processor' лучше оставить, если он нужен дальше
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
else:
    print("Пропуск запуска INT8 ONNX на GPU из-за отсутствия провайдера, папки модели или onnxruntime.")
    stats_int8_gpu = {"error": "Пропущено из-за зависимостей"}

print("Завершено INT8 ONNX на GPU.")


--- Запуск INT8 ONNX на GPU ---
Доступные провайдеры ONNX Runtime: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Загрузка INT8 ONNX модели из whisper_onnx_int8 на GPU...


2025-05-13 02:53:45.542375038 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 768 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-05-13 02:53:45.566808740 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-13 02:53:45.566835732 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2025-05-13 02:53:50.774461247 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 1348 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact

VRAM занято после загрузки INT8 ONNX модели: 2953.0 MB
Создание пайплайна для INT8 ONNX на GPU...
Запуск оценки INT8 ONNX на GPU...
Оценка модели (_ORTModelForWhisper) на устройстве: cuda:0...
Статистика пиковой VRAM PyTorch сброшена.
Пиковая VRAM по статистике PyTorch: 3484.3 MB

Результаты INT8 ONNX GPU: {'wer': 0.02390438247011952, 'latency_s': 7.1746272325515745, 'processed_items': 10, 'peak_ram_mb': '832.4', 'peak_vram_mb': '3484.3'}
Завершено INT8 ONNX на GPU.
